In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import load_cifar

In [2]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [3]:
val_features, val_labels = pickle.load(open('val_data.pickle','rb'))
train_features,train_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))
val_labels = np.squeeze(val_labels.todense())
train_labels = np.squeeze(train_labels.todense())
test_labels = np.squeeze(test_labels.todense())

#learning rate
lr = 0.001
#number of traning epochs
epochs = 10
#number of batch_size
batch_size = 32
total_batch = int(40000/batch_size)
num_steps = (epochs+1) * total_batch

#network parameters
n_hidden_1 = 800
n_hidden_2 = 400
n_hidden_3 = 200
n_hidden_4 = 100
n_hidden_5 = 60
n_hidden_6 = 30
num_input = 3072
num_classes = 10

<h1>Define Placeholder</h1>

In [4]:
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1],stddev=0.1),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=0.1),name='W2'),
    'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=0.1), name='W3'),
    'W4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=0.1), name='W4'),
    'W5': tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5],stddev=0.1), name='W5'),
    'W6': tf.Variable(tf.random_normal([n_hidden_5, n_hidden_6],stddev=0.1), name='W6'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_6, num_classes],stddev=0.1),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'b4': tf.Variable(tf.zeros(shape=[n_hidden_4]),name='b4'),
    'b5': tf.Variable(tf.zeros(shape=[n_hidden_5]), name='b5'),
    'b6': tf.Variable(tf.zeros(shape=[n_hidden_6]), name='b6'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

<h1>Define Neural Network Architecture</h1>

In [5]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.nn.relu(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
    layer_2_out = tf.nn.relu(tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']))
    layer_3_out = tf.nn.relu(tf.add(tf.matmul(layer_2_out, weights['W3']), biases['b3']))
    layer_4_out = tf.nn.relu(tf.add(tf.matmul(layer_3_out, weights['W4']), biases['b4']))
    layer_5_out = tf.nn.relu(tf.add(tf.matmul(layer_4_out, weights['W5']), biases['b5']))
    layer_6_out = tf.nn.relu(tf.add(tf.matmul(layer_5_out, weights['W6']), biases['b6']))
    out = tf.add(tf.matmul(layer_6_out,weights['Wout']),biases['bout'])
    return out

#predicted labels
logits = neural_net(X)
Y_hat = tf.nn.softmax(logits)

<h1>Define cost andoptimization</h1>

In [6]:
#define cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='cost')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(Y_hat,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [7]:
def next_batch(batch_size):
    idx = np.arange(0 , len(train_features))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [train_features[i] for i in idx]
    labels_shuffle = [train_labels[i] for i in idx]
    data_shuffle = np.asarray(data_shuffle)
    labels_shuffle = np.asarray(labels_shuffle)
    labels_shuffle = np.squeeze(labels_shuffle)
    return data_shuffle, labels_shuffle

In [11]:
with tf.Session() as sess:
    sess.run(init)
    # Timer
    time_start = time.time()
    for i in range(num_steps):
        # fetch batch
        batch_x, batch_y = next_batch(batch_size)
        # run optimization
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if (i > 1) and (i % total_batch == 0):
            time_end = time.time()
            acc = sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
            print("Epoch " + str(i/total_batch) + ", Accuracy: {:.3f}".format(acc)+ ", Time: {:.3f}\t".format(time_end-time_start))
            print("Validation Accuracy:{:.3f}".format(float(sess.run(accuracy, feed_dict={X: val_features, Y: val_labels}))))
            time_start = time.time()

    print("Training finished!")

    print("Testing Accuracy::{:.3f}".format(float(sess.run(accuracy, feed_dict={X: test_features, Y: test_labels}))))

Epoch 1.0, Accuracy: 0.250, Time: 9.264	
Validation Accuracy:0.274
Epoch 2.0, Accuracy: 0.406, Time: 9.280	
Validation Accuracy:0.362
Epoch 3.0, Accuracy: 0.344, Time: 7.991	
Validation Accuracy:0.387
Epoch 4.0, Accuracy: 0.500, Time: 9.069	
Validation Accuracy:0.429
Epoch 5.0, Accuracy: 0.469, Time: 7.706	
Validation Accuracy:0.435
Epoch 6.0, Accuracy: 0.750, Time: 6.266	
Validation Accuracy:0.461
Epoch 7.0, Accuracy: 0.500, Time: 6.737	
Validation Accuracy:0.484
Epoch 8.0, Accuracy: 0.500, Time: 7.144	
Validation Accuracy:0.474
Epoch 9.0, Accuracy: 0.500, Time: 8.881	
Validation Accuracy:0.513
Epoch 10.0, Accuracy: 0.531, Time: 9.125	
Validation Accuracy:0.517
Training finished!
Testing Accuracy::0.519
